In [81]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import matplotlib as plt
from vnstock import *

from typing import Literal


Theo payback time. Để định giá một cổ phiếu thì cần các chỉ số tài chính sau có mức tăng trưởng trên 10%
- Lợi nhuận sau thuế
- EPS: earningPerShare
- PE < 10: priceToEarning
- BVPS: bookValuePerShare
- ROE: roe
- Cash hoạt động kinh doanh > 0
- Tổng Nợ < 3 Sum (Lợi nhuận 3 năm)

In [104]:
class PayBackTime:
    def __init__(self, symbol: str = 'GEX',
                 report_range: Literal['yearly', 'quarterly'] = 'yearly',
                 is_all: bool = True,
                 window_size:int = 10,):

        self.window_size = window_size
        self.symbol = symbol
        self.report_range = report_range
        self.is_all = is_all
        # Initialize indicator_df, income_df, and balance_df using class variables
        self.indicator_df = financial_ratio(symbol=self.symbol, report_range=self.report_range, is_all=self.is_all).T.reset_index() 
        self.income_df = financial_flow(symbol=self.symbol, report_type='incomestatement', report_range=self.report_range).reset_index()
        self.balance_df = financial_flow(symbol=self.symbol, report_type='balancesheet', report_range=self.report_range).reset_index()
        self.interest_cols = ['roe', 'earningPerShare', 'bookValuePerShare', 'revenue', 'grossProfit', 'capital']

    def calculate_interest_rate(self, df, column_name: str):
        # Consider the first 10 rows using iloc
        selected_rows = df.head(self.window_size)
        
        # Calculate interest using the formula: (current_value - initial_value) / initial_value * 100
        initial_value = selected_rows[column_name].iloc[-1]
        current_value = selected_rows[column_name].iloc[0]
        # interest = ((current_value - initial_value) / initial_value) * 100
        time_length = selected_rows[column_name].count()
        interest =  ((current_value / initial_value) ** (1 / time_length) - 1) * 100
        
        if interest > 10:
            status = 'Good'
        elif interest < 0:
            status = 'Really Bad'
        else:
            status = 'Bad'
        
        # Print the result
        print(f'The {column_name}: {interest:.2f}% through {time_length} {self.report_range}: {status}')
        
        # Check if interest is greater than 10% and print the corresponding message

        return interest
    
    def estimate_mean_interest(self, col_list: list ):
        mean_interest = []
        for col in col_list:
            interest = self.calculate_interest_rate(self.get_dataframe_by_column(col), column_name=col)
            mean_interest.append(interest)
        mean_interest = np.mean(mean_interest)
        return mean_interest
    
    def get_dataframe_by_column(self, column_name: str):
        """
        Get the DataFrame that contains the specified column.
        """
        dataframes = [self.indicator_df, self.income_df, self.balance_df]
        for df in dataframes:
            if column_name in df.columns:
                return df
        return None
    
    def check_debt(self):
        debt_col = 'debt'
        profit_col = 'grossProfit'
        debt = self.balance_df[debt_col].iloc[0]
        profit = self.income_df[profit_col].iloc[:2].sum()
        
        print(f'Debt: {debt} - Profit: {profit}')
        
        if debt > profit:
            print('Debt is more than profit.')
        elif debt < profit:
            print('Profit is more than debt.')
        else:
            print('Debt and profit are equal.')
    
    def calculate_price(self, current_eps=None, future_pe=None, future_growth_rate=None,
                        MARR: int = 15, NoY: int = 10, MOS: int = 50):
        if current_eps is None:
            # The current_eps for yearly is 1-year lag
            indicator_df_temp = financial_ratio(symbol=self.symbol, report_range='quarterly', is_all=False).T.reset_index() 
            current_eps = indicator_df_temp['earningPerShare'].iloc[0]
        if future_pe is None:
            future_pe = self.indicator_df['priceToEarning'].mean()
        if future_growth_rate is None:
            future_growth_rate = self.estimate_mean_interest(col_list=self.interest_cols)

        # Convert the percentage rates to decimals
        MARR /= 100
        MOS /= 100
        future_growth_rate /= 100

        # Calculate the future EPS using the compound annual growth rate formula
        future_eps = current_eps * (1 + future_growth_rate) ** NoY
        # Calculate the future price using the future PE and future EPS
        future_price = future_pe * future_eps
        # Calculate the present value using the discounted cash flow formula
        present_value = future_price / (1 + MARR) ** NoY
        # Print the result
        print(f"\nCurrent EPS: {current_eps}")
        print(f"Future PE: {future_pe:.2f}")
        print(f"Future Growth Rate: {future_growth_rate * 100}%")
        print(f"Discount Rate (MARR): {MARR * 100}%")
        print(f"Number of Years: {NoY}")
        print(f"MOS Multiplier: {MOS * 100}%")
        print("-------------------------")
        print(f"{self.symbol} - Sticker price: {present_value/1000:.2f} - MOS price: {(present_value * MOS)/1000:.2f} (nghìn VND)")


In [108]:
pbt_generator = PayBackTime(symbol='VIC', report_range='yearly', window_size=10)

In [109]:
pbt_generator.check_debt()

Debt: 441752 - Profit: 48759
Debt is more than profit.


In [110]:
pbt_generator.calculate_price()

The roe: -17.11% through 10 yearly: Really Bad
The earningPerShare: 2.60% through 10 yearly: Bad
The bookValuePerShare: 26.68% through 10 yearly: Good
The revenue: 18.67% through 10 yearly: Good
The grossProfit: 7.65% through 10 yearly: Bad
The capital: 15.33% through 10 yearly: Good

Current EPS: 991
Future PE: 30.30
Future Growth Rate: 8.968695951251403%
Discount Rate (MARR): 15.0%
Number of Years: 10
MOS Multiplier: 50.0%
-------------------------
VIC - Sticker price: 17.52 - MOS price: 8.76 (nghìn VND)
